In [1]:
# install boto3 for AWS connection
dbutils.library.installPyPI("boto3", version="1.9.157")
dbutils.library.restartPython()

In [2]:
# install these Maven packages in the cluster
# JohnSnowLabs:spark-nlp:2.3.4

In [3]:
# import AWS credentials
# import config.py ##for local
%run "/dbfs/FileStore/tables/config" ##for databricks

In [4]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [5]:
# get or create Spark session
app_name = "spark-airbnb-sentiment"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [6]:
import boto3

secret_name = my_secret_name
region_name = my_region_name
access_key  = my_access_key
secret_key  = my_secret_key

session      = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
client       = session.client('secretsmanager')
secret_value = client.get_secret_value(SecretId=secret_name)

In [7]:
import json
def get_connection(secret_value):
    return json.loads(secret_value['SecretString'])

In [8]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort     = connection['port']
jdbcDatabase = "postgres"
dialect      = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user"     : jdbcUsername,
  "password" : jdbcPassword,
  "driver"   : "org.postgresql.Driver" 
}

In [9]:
# Read from reviews_full table

table = "reviews_full"

reviews_df = spark.read.jdbc(url=jdbcUrl, table=table, properties=connectionProperties)
reviews_df.printSchema()

root
-- listing_id: integer (nullable = true)
-- id: integer (nullable = true)
-- date: date (nullable = true)
-- reviewer_id: integer (nullable = true)
-- reviewer_name: string (nullable = true)
-- comments: string (nullable = true)

In [10]:
from pyspark.ml import Pipeline
import sparknlp
sparknlp.start()
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

pipeline = PretrainedPipeline("analyze_sentiment", lang="en")

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [11]:
thousand_row = reviews_df.withColumnRenamed("comments", "text").limit(1000)

In [12]:
annotations_df = pipeline.transform(thousand_row)
# display(annotations_df)

In [13]:
# return the sentiment with the highest confidence value
@F.udf()
def highestConfidence(sentClassifications):
  idx = np.argmax([x["metadata"]["confidence"] for x in sentClassifications])
  return sentClassifications[idx]["result"]


# return sentiment of the longest sentence
@F.udf()
def longestSentiment(sentClassifications):
  sentLengths = [
      [x["result"], x["end"] - x["begin"]]
    for x in sentClassifications
  ]
  posSent = sum([x[1] for x in sentLengths if x[0] == "positive"])
  negSent = sum([x[1] for x in sentLengths if x[0] == "negative"])
  return "positive" if posSent>negSent else "negative"


# return the overall average sentiment by votes
@F.udf()
def averageSentiment(sentClassifications):
  sentConf = [
      [x["result"], x["metadata"]["confidence"]]
    for x in sentClassifications
  ]
  posSent = np.mean([float(x[1]) for x in sentConf if x[0] == "positive"])
  negSent = np.mean([float(x[1]) for x in sentConf if x[0] == "negative"])
  return "positive" if posSent>=negSent else "negative"


# return the overall average sentiment after normalizing by sentence length
@F.udf()
def classifySentiment(sentClassifications):
  
  sentConf = [
      [x["result"], x["metadata"]["confidence"]]
    for x in sentClassifications
  ]
  posSentConf = np.mean([float(x[1]) for x in sentConf if x[0] == "positive"])
  negSentConf = np.mean([float(x[1]) for x in sentConf if x[0] == "negative"])
  
  sentLengths = [
      [x["result"], x["end"] - x["begin"]]
    for x in sentClassifications
  ]
  posSentLengths = sum([x[1] for x in sentLengths if x[0] == "positive"])
  negSentLengths = sum([x[1] for x in sentLengths if x[0] == "negative"])
  
  # set to NaN if we have null comments
  if posSentLengths == 0 and negSentLengths == 0:
    return np.nan
  
  if abs(posSentLengths - negSentLengths)/(posSentLengths + negSentLengths) > 0.2:
    return "positive" if posSentLengths>negSentLengths else "negative"
  
  else:
    return "positive" if posSentConf>negSentConf else "negative"

In [14]:
annotations_df.printSchema()

root
-- listing_id: integer (nullable = true)
-- id: integer (nullable = true)
-- date: date (nullable = true)
-- reviewer_id: integer (nullable = true)
-- reviewer_name: string (nullable = true)
-- text: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- checked: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentiment: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [15]:
@F.udf()
def example(sentiment):
  print(sentiment)

In [16]:
display(annotations_df)

listing_id id date reviewer_id reviewer_name text document sentence token checked sentiment 5456 16489 2009-11-07 46119 Kevin Very accommodating, great space. List(List(document, 0, 31, Very accommodating, great space., Map(sentence -> 0), List())) List(List(document, 0, 31, Very accommodating, great space., Map(sentence -> 0), List())) List(List(token, 0, 3, Very, Map(sentence -> 0), List()), List(token, 5, 17, accommodating, Map(sentence -> 0), List()), List(token, 18, 18, ,, Map(sentence -> 0), List()), List(token, 20, 24, great, Map(sentence -> 0), List()), List(token, 26, 30, space, Map(sentence -> 0), List()), List(token, 31, 31, ., Map(sentence -> 0), List())) List(List(token, 0, 3, Very, Map(confidence -> 1.0), List()), List(token, 5, 17, accommodating, Map(confidence -> 1.0), List()), List(token, 18, 18, ,, Map(confidence -> 0.0), List()), List(token, 20, 24, great, Map(confidence -> 1.0), List()), List(token, 26, 30, space, Map(confidence -> 1.0), List()), List(token, 31, 31, ., Map(confidence -> 0.0), List())) List(List(sentiment, 0, 31, negative, Map(confidence -> 0.5582), List())) 5456 18215 2009-11-24 54243 Jane 6th Street is a bit of a walk but location is central to lots of shopping, nightlife and eateries. Place is clean, private and offers an array of videos and dcs. Great coffee, coffee pot and Fiesta Ware in the kitchen. List(List(document, 0, 217, 6th Street is a bit of a walk but location is central to lots of shopping, nightlife and eateries. Place is clean, private and offers an array of videos and dcs. Great coffee, coffee pot and Fiesta Ware in the kitchen., Map(sentence -> 0), List())) List(List(document, 0, 97, 6th Street is a bit of a walk but location is central to lots of shopping, nightlife and eateries., Map(sentence -> 0), List()), List(document, 99, 160, Place is clean, private and offers an array of videos and dcs., Map(sentence -> 1), List()), List(document, 162, 217, Great coffee, coffee pot and Fiesta Ware in the kitchen., Map(sentence -> 2), List())) List(List(token, 0, 2, 6th, Map(sentence -> 0), List()), List(token, 4, 9, Street, Map(sentence -> 0), List()), List(token, 11, 12, is, Map(sentence -> 0), List()), List(token, 14, 14, a, Map(sentence -> 0), List()), List(token, 16, 18, bit, Map(sentence -> 0), List()), List(token, 20, 21, of, Map(sentence -> 0), List()), List(token, 23, 23, a, Map(sentence -> 0), List()), List(token, 25, 28, walk, Map(sentence -> 0), List()), List(token, 30, 32, but, Map(sentence -> 0), List()), List(token, 34, 41, location, Map(sentence -> 0), List()), List(token, 43, 44, is, Map(sentence -> 0), List()), List(token, 46, 52, central, Map(sentence -> 0), List()), List(token, 54, 55, to, Map(sentence -> 0), List()), List(token, 57, 60, lots, Map(sentence -> 0), List()), List(token, 62, 63, of, Map(sentence -> 0), List()), List(token, 65, 72, shopping, Map(sentence -> 0), List()), List(token, 73, 73, ,, Map(sentence -> 0), List()), List(token, 75, 83, nightlife, Map(sentence -> 0), List()), List(token, 85, 87, and, Map(sentence -> 0), List()), List(token, 89, 96, eateries, Map(sentence -> 0), List()), List(token, 97, 97, ., Map(sentence -> 0), List()), List(token, 99, 103, Place, Map(sentence -> 1), List()), List(token, 105, 106, is, Map(sentence -> 1), List()), List(token, 108, 112, clean, Map(sentence -> 1), List()), List(token, 113, 113, ,, Map(sentence -> 1), List()), List(token, 115, 121, private, Map(sentence -> 1), List()), List(token, 123, 125, and, Map(sentence -> 1), List()), List(token, 127, 132, offers, Map(sentence -> 1), List()), List(token, 134, 135, an, Map(sentence -> 1), List()), List(token, 137, 141, array, Map(sentence -> 1), List()), List(token, 143, 144, of, Map(sentence -> 1), List()), List(token, 146, 151, videos, Map(sentence -> 1), List()), List(token, 153, 155, and, Map(sentence -> 1), List()), List(token, 157, 159, dcs, Map(sentence -> 1), List()), List(token, 160, 160, ., Map(sentence -> 1), List()), List(token, 162, 166, Great, Ma

In [17]:
annotations_df.limit(1).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-775457500887042> in <module> 
 ----> 1 annotations_df . limit ( 1 ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 379 """
 380 if isinstance ( truncate , bool ) and truncate : 
 --> 381 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 382 else : 
 383 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1651.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 32.0 failed 1 times, most recent failure: Lost task 0.0 in stage 32.0 (TID 60, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$dfAssembleNoExtras$1: (string) => array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:148)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:533)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:539)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.NullPointerException
	at com.johnsnowlabs.nlp.DocumentAssembler.assemble(DocumentAssembler.scala:98)
	at com.johnsnowlabs.nlp.DocumentAssembler$$anonfun$dfAssembleNoExtras$1.apply(DocumentAssembler.scala:124)
	at com.johnsnowlabs.nlp.DocumentAssembler$$anonfun$dfAssembleNoExtras$1.apply(DocumentAssembler.scala:123)
	... 15 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2350)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2349)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abor

In [18]:
# sentiment_df = annotations_df.select("listing_id", "id", "date", "reviewer_id", "reviewer_name", "text", "sentiment",
#                                      averageSentiment("sentiment").alias("avg_sentiment"),
#                                      highestConfidence("sentiment").alias("high_conf_sentiment"),
#                                      longestSentiment("sentiment").alias("long_conf_sentiment"),
#                                      classifySentiment("sentiment").alias("classified_sentiment")
#                                     )
# display(sentiment_df)

annotations_df.limit(1).select("listing_id", "id", "date", "reviewer_id", "reviewer_name", "text", "sentiment",
                                     longestSentiment("sentiment").alias("longest_sentiment")
                                    ).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2519330511902427> in <module> 
 8 
 9 annotations_df.limit(1).select("listing_id", "id", "date", "reviewer_id", "reviewer_name", "text", "sentiment",
 ---> 10 longestSentiment ( "sentiment" ) . alias ( "longest_sentiment" ) 
 11 ).show()

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 379 """
 380 if isinstance ( truncate , bool ) and truncate : 
 --> 381 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 382 else : 
 383 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1275.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 57, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$dfAssembleNoExtras$1: (string) => array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:148)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:533)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:539)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.NullPointerException
	at com.johnsnowlabs.nlp.DocumentAssembler.assemble(DocumentAssembler.scala:98)
	at com.johnsnowlabs.nlp.DocumentAssembler$$anonfun$dfAssembleNoExtras$1.apply(DocumentAssembler.scala:124)
	at com.johnsnowlabs.nlp.DocumentAssembler$$anonfun$dfAssembleNoExtras$1.apply(DocumentAssembler.scala:123)
	... 15 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2350)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2349)
	at scala.collection.mutable.ResizableArray$

In [19]:
avg_sentiment_df = sentiment_df.groupBy("listing_id").pivot("avg_sentiment", ["positive", "negative"]).count().orderBy("listing_id")
avg_sentiment_df = avg_sentiment_df.withColumnRenamed("positive", "avg_sentiment_positive_count") \
                                   .withColumnRenamed("negative", "avg_sentiment_negative_count") \
                                   .withColumn("avg_sentiment_positive_percent", F.expr("CASE WHEN avg_sentiment_negative_count is null and avg_sentiment_positive_count > 0 THEN 1 WHEN avg_sentiment_positive_count is null and avg_sentiment_negative_count > 0 THEN 0 ELSE avg_sentiment_positive_count/(avg_sentiment_positive_count + avg_sentiment_negative_count) END")) \
                                   .withColumn("avg_sentiment_positivity", F.expr("CASE WHEN avg_sentiment_positive_percent > 0.7 THEN 1 ELSE 0 END"))

In [20]:
high_conf_sentiment_df = sentiment_df.groupBy("listing_id").pivot("high_conf_sentiment", ["positive", "negative"]).count().orderBy("listing_id")
high_conf_sentiment_df = high_conf_sentiment_df.withColumnRenamed("positive", "high_conf_sentiment_positive_count") \
                                               .withColumnRenamed("negative", "high_conf_sentiment_negative_count") \
                                               .withColumn("high_conf_sentiment_positive_percent", F.expr("CASE WHEN high_conf_sentiment_negative_count is null and high_conf_sentiment_positive_count > 0 THEN 1 WHEN high_conf_sentiment_positive_count is null and high_conf_sentiment_negative_count > 0 THEN 0 ELSE high_conf_sentiment_positive_count/(high_conf_sentiment_positive_count + high_conf_sentiment_negative_count) END")) \
                                               .withColumn("high_conf_sentiment_positivity", F.expr("CASE WHEN high_conf_sentiment_positive_percent > 0.7 THEN 1 ELSE 0 END"))

In [21]:
long_conf_sentiment_df = sentiment_df.groupBy("listing_id").pivot("long_conf_sentiment", ["positive", "negative"]).count().orderBy("listing_id")
long_conf_sentiment_df = long_conf_sentiment_df.withColumnRenamed("positive", "long_conf_sentiment_positive_count") \
                                               .withColumnRenamed("negative", "long_conf_sentiment_negative_count") \
                                               .withColumn("long_conf_sentiment_positive_percent", F.expr("CASE WHEN long_conf_sentiment_negative_count is null and long_conf_sentiment_positive_count > 0 THEN 1 WHEN long_conf_sentiment_positive_count is null and long_conf_sentiment_negative_count > 0 THEN 0 ELSE long_conf_sentiment_positive_count/(long_conf_sentiment_positive_count + long_conf_sentiment_negative_count) END")) \
                                               .withColumn("long_conf_sentiment_positivity", F.expr("CASE WHEN long_conf_sentiment_positive_percent > 0.7 THEN 1 ELSE 0 END"))

In [22]:
class_sentiment_df = sentiment_df.groupBy("listing_id").pivot("classified_sentiment", ["positive", "negative"]).count().orderBy("listing_id")
class_sentiment_df = class_sentiment_df.withColumnRenamed("positive", "classified_sentiment_positive_count") \
                                       .withColumnRenamed("negative", "classified_sentiment_negative_count") \
                                       .withColumn("classified_sentiment_positive_percent", F.expr("CASE WHEN classified_sentiment_negative_count is null and classified_sentiment_positive_count> 0 THEN 1 WHEN classified_sentiment_positive_count is null and classified_sentiment_negative_count > 0 THEN 0 ELSE classified_sentiment_positive_count/(classified_sentiment_positive_count + classified_sentiment_negative_count) END")) \
                                       .withColumn("classified_sentiment_positivity", F.expr("CASE WHEN classified_sentiment_positive_percent > 0.7 THEN 1 ELSE 0 END"))

In [23]:
all_sentiment_df = avg_sentiment_df.join(high_conf_sentiment_df, ["listing_id"], how="inner") \
                                   .join(long_conf_sentiment_df, ["listing_id"], how="inner") \
                                   .join(class_sentiment_df, ["listing_id"], how="inner")

In [24]:
save_mode = "overwrite"

all_sentiment_df.write \
    .format("com.databricks.spark.csv") \
    .options(header="true") \
    .mode(save_mode) \
    .save("/FileStore/tables/all_sentiment.csv")

In [25]:
display(dbutils.fs.ls("/FileStore/tables"))

path,name,size
dbfs:/FileStore/tables/airlines.csv,airlines.csv,505
dbfs:/FileStore/tables/all_sentiment.csv/,all_sentiment.csv/,0
dbfs:/FileStore/tables/amazon_reviews_us_Digital_Video_Games_v1_00_tsv-5da15.gz,amazon_reviews_us_Digital_Video_Games_v1_00_tsv-5da15.gz,27442648
dbfs:/FileStore/tables/amazon_reviews_us_Software_v1_00_tsv-16dc0.gz,amazon_reviews_us_Software_v1_00_tsv-16dc0.gz,94010685
dbfs:/FileStore/tables/amazon_reviews_us_Video_Games_v1_00_tsv-56dc1.gz,amazon_reviews_us_Video_Games_v1_00_tsv-56dc1.gz,475199894
dbfs:/FileStore/tables/austin_weather_2017.csv,austin_weather_2017.csv,609844
dbfs:/FileStore/tables/calendar.csv,calendar.csv,180039751
dbfs:/FileStore/tables/config.py,config.py,174
dbfs:/FileStore/tables/customer_data.csv/,customer_data.csv/,0
dbfs:/FileStore/tables/data.csv,data.csv,73


In [26]:
listings_df = spark.read.jdbc(url=jdbcUrl, table="listings_full", properties=connectionProperties)
listings_df = listings_df.select("id", "latitude", "longitude", "review_scores_rating")

In [27]:
final_df = all_sentiment_df.join(listings_df, all_sentiment_df.listing_id == listings_df.id, how="inner").drop(F.col("id"))
display(final_df)

listing_id,avg_sentiment_positive_count,avg_sentiment_negative_count,avg_sentiment_positive_percent,avg_sentiment_positivity,high_conf_sentiment_positive_count,high_conf_sentiment_negative_count,high_conf_sentiment_positive_percent,high_conf_sentiment_positivity,long_conf_sentiment_positive_count,long_conf_sentiment_negative_count,long_conf_sentiment_positive_percent,long_conf_sentiment_positivity,classified_sentiment_positive_count,classified_sentiment_negative_count,classified_sentiment_positive_percent,classified_sentiment_positivity,latitude,longitude,review_scores_rating
21368263,17,14,0.5483870967741935,0,28,3,0.9032258064516129,1,22,9,0.7096774193548387,1,23,8,0.7419354838709677,1,30.2936,-97.6995,97
27310107,1,2,0.3333333333333333,0,2,1,0.6666666666666666,0,2,1,0.6666666666666666,0,2,1,0.6666666666666666,0,30.3841,-97.699,98
17611669,null,3,0.0,0,1,2,0.3333333333333333,0,2,1,0.6666666666666666,0,2,1,0.6666666666666666,0,30.2864,-97.7489,90
21378172,1,null,1.0,1,1,null,1.0,1,1,null,1.0,1,1,null,1.0,1,30.2547,-97.7766,100
21364411,3,1,0.75,1,4,null,1.0,1,3,1,0.75,1,4,null,1.0,1,30.2204,-97.9106,100
22937302,null,1,0.0,0,1,null,1.0,1,1,null,1.0,1,1,null,1.0,1,30.2581,-97.7283,100
27102313,1,2,0.3333333333333333,0,2,1,0.6666666666666666,0,1,2,0.3333333333333333,0,1,2,0.3333333333333333,0,30.4362,-97.7307,98
21366577,12,11,0.5217391304347826,0,22,1,0.9565217391304348,1,20,3,0.8695652173913043,1,20,3,0.8695652173913043,1,30.1469,-97.8568,98
977492,32,20,0.6153846153846154,0,43,9,0.8269230769230769,1,36,16,0.6923076923076923,0,36,16,0.6923076923076923,0,30.2521,-97.7238,98
21380835,1,1,0.5,0,2,null,1.0,1,1,1,0.5,0,1,1,0.5,0,30.2661,-97.7426,98
